In [8]:
import os
import re
import rasterio
import xarray as xr
import geopandas as gpd
import pandas as pd
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_bounds
from affine import Affine

In [17]:
# Ruta de la carpeta que contiene los archivos NetCDF
nc_folder = '../DataIntermediate/2m_temperature'
# Ruta de la carpeta para guardar los archivos TIF convertidos
tif_folder = '../DataIntermediate/2m_temperature_tif'
# Archivo shapefile de los centroides
centroid_shapefile = '../DataIn/Centroides.shp'
# Archivo Excel de salida
output_excel = '../Results/temperaturas_centroides.xlsx'



In [ ]:
# Leer el shapefile de los centroides
centroids = gpd.read_file(centroid_shapefile)

# Lista para almacenar los resultados
results = []

In [20]:
# Función para extraer la fecha del nombre del archivo usando regex
def extract_date_from_filename(filename):
    match = re.search(r'(\d{8})', filename)
    if match:
        return match.group(1)
    return None

# Extraer valores de temperatura en los centroides directamente de los archivos .nc
for nc_file in os.listdir(nc_folder):
    if nc_file.endswith('.nc'):
        nc_path = os.path.join(nc_folder, nc_file)
        date = extract_date_from_filename(nc_file)
        
        # Abrir el archivo NetCDF
        ds = xr.open_dataset(nc_path)
        var_name = 'Temperature_Air_2m_Mean_Day_Time'  # Nombre de la variable en el archivo NetCDF (ajusta según corresponda)
        data = ds[var_name]

        # Iterar sobre los centroides
        for index, row in centroids.iterrows():
            centroid_geom = row['geometry']
            centroid_x = centroid_geom.x
            centroid_y = centroid_geom.y

            # Extraer el valor de temperatura en el centroide
            temp_value = data.sel(lon=centroid_x, lat=centroid_y, method='nearest').values.item()
            
            # Añadir el resultado a la lista
            results.append({
                'Date': date,
                'Centroid_X': centroid_x,
                'Centroid_Y': centroid_y,
                'Temperature': temp_value
            })

# Crear un DataFrame de pandas con los resultados
df = pd.DataFrame(results)

# Guardar el DataFrame en un archivo Excel
df.to_excel(output_excel, index=False)